# Cleaning data and creating custom features for Give Me Some Credit Kaggle Challenge

1. Cleaning the monthly income from NA values
2. Clean the debt ratio by replacing NA by the mean (Should ideally be done after splitting and the mean should be calculated on the training set if you want to do an evaluation)
3. Creating a montlhy debt feature
    * monthly income multiplied by debt ratio if income is not 0
    * debt ratio if income is 0
4. Create a Balanced Income feature that take into account Income and debt ratio
    * Set income to 0 when negative.
5. Clean the number of dependents feature
    * set NA to zero
6. Create a Blanced Income per household members feature
7. Cleaning the Number of Times Late feature
    * Create a custom categorical feature that contains 2 different tags for each row that contains a Number of time late of either 96 or 98
    * Remove the 96 and 98 values (Replacing those values by NA or some other justifiable value)
8. Add a feature that compute the weighted sum of the number of time late per duration
    * weight of 3 for 90 days and more
    * weight of 2 for 60 to 89 days
    * weight of 1 for 30 to 59 days


In [574]:
from pandas import DataFrame, read_csv, to_numeric
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from bigml.api import BigML
import kaggle

%matplotlib inline

In [ ]:
compression = True  #Do we compress the csv files?

version='v1.3'

In [ ]:
suffix = version + '.csv'
if compression:
    suffix +='.bz2'

filename_fulltrain = 'gmsc-fulltrain-' + suffix
filename_train = 'gmsc-train80-' + suffix
filename_valid = 'gmsc-valid20-' + suffix
filename_test = 'gmsc-test-' + suffix

filename_bigml_ids = 'bigml-' + suffix

## Loading csv files as data frames

Files must be placed in the same directory as this file. Alternatively, modify the relative path to those files.

In [ ]:
!pwd
!ls
fulltrain=read_csv('./cs-training.csv',index_col=0)
test=read_csv('./cs-test.csv',index_col=0)

## Reordering the objective field column to the end

In [ ]:
fulltrain=fulltrain[[c for c in fulltrain if c != 'SeriousDlqin2yrs']+['SeriousDlqin2yrs']]
test=test[[c for c in test if c != 'SeriousDlqin2yrs']+['SeriousDlqin2yrs']]

In [ ]:
train80, test20 = train_test_split(fulltrain, test_size=0.2)
data_sets=[fulltrain,train80,test20,test]
filenames=[filename_fulltrain,filename_train,filename_valid,filename_test]

## Correcting the values in monthly income
Set NaN to 0 in the monthly income column

In [ ]:
help(DataFrame.fillna)

In [ ]:
for df in data_sets:
    df.loc[:,'MonthlyIncome'].fillna(0, inplace=True)
    
data_sets[0].head(10)

## Cleaning Debt ratio
Set NA values to the mode from the training set

In [ ]:
mode_debt_ratio=train80['DebtRatio'].mode

for df in data_sets:
    df.loc[:,'DebtRatio'].fillna(mode_debt_ratio, inplace=True)
    
data_sets[0].head(20)

## Creating a new column for monthly debt.

In [ ]:
help(DataFrame.insert)

In [ ]:
for df in data_sets:
    df.insert(5,"MonthlyDebt",0)
data_sets[0].head(10)

If there is no income set DebtRatio to the mean debt ratio.

In [ ]:
for df in data_sets:
    df.loc[:,'MonthlyDebt']=df['DebtRatio']*df['MonthlyIncome']
    df.loc[df['MonthlyIncome'] == 0,'MonthlyDebt']=df.loc[df['MonthlyIncome'] == 0,'DebtRatio']
    #df.loc[df['MonthlyIncome'] == 0,'AbsoluteDebt']=df.loc[df['MonthlyIncome'] == 0,'DebtRatio'] #0.86222
    
    #df.loc[df['MonthlyIncome'] == 0,'DebtRatio']=0 #0.85533
    df.loc[df['MonthlyIncome'] == 0,'DebtRatio']=None #0.8610
    #df.loc[df['MonthlyIncome'] == 0,'DebtRatio']=mode_debt_ratio #0.8517

data_sets[0][['MonthlyDebt','DebtRatio','MonthlyIncome']].head(10)

## Creating a balanced Income feature

In [ ]:
for df in data_sets:
    #df.insert(6,'BalancedIncome',0)
    df.insert(6,'BalancedIncome',None)

In [ ]:
for df in data_sets:
    income_positive = df['MonthlyIncome'] > df['MonthlyDebt']
    df.loc[income_positive,'BalancedIncome']= df.loc[income_positive,'MonthlyIncome'] - df.loc[income_positive,'MonthlyDebt']

data_sets[0].head(20)

## Cleaning the number of dependents column
Set to 0 the number of dependents when not available.

In [ ]:
for df in data_sets:
    df.loc[:,'NumberOfDependents'].fillna(0, inplace=True)
    df.loc[:,'NumberOfDependents']=to_numeric(df['NumberOfDependents'],downcast='integer')
    
data_sets[0][['NumberOfDependents']].head(10)

## Add a balanced income per household members feature

In [ ]:
for df in data_sets:
    df.insert(7,'IncomePerHouseholdMember',0)

In [ ]:
for df in data_sets:
    df.loc[:,'IncomePerHouseholdMember']= df['BalancedIncome'] / (df['NumberOfDependents']+1)
data_sets[0][['IncomePerHouseholdMember']].head(20)

## Adding a "number of times late" categorical feature

In [ ]:
for df in data_sets:
    df.insert(8,'LateCategory',0)

In [ ]:
data_sets[0].loc[data_sets[0]['NumberOfTimes90DaysLate']==98,['LateCategory','NumberOfTimes90DaysLate']].head(20)

In [ ]:
for df in data_sets:
    df.loc[:,'LateCategory']="L0"
    df.loc[df['NumberOfTimes90DaysLate'] == 98,'LateCategory']="L98"
    df.loc[df['NumberOfTimes90DaysLate'] == 96,'LateCategory']="L96"
    df.loc[(df['NumberOfTimes90DaysLate'] == 98) | (df['NumberOfTimes90DaysLate'] == 96),
              ['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate']]=None
              
data_sets[0].loc[(data_sets[0]['LateCategory']=='L96') | (data_sets[0]['LateCategory']=='L98'),['LateCategory','NumberOfTimes90DaysLate']].head(20)

## Add a "Late score" feature

In [ ]:
for df in data_sets:
    df.insert(0,'LateScore',0)

In [ ]:
weight=[1,2,3]
#weight=[1,2,4]
for df in data_sets:
    df.loc[:,'LateScore']=weight[2]*df['NumberOfTimes90DaysLate']+weight[1]*df['NumberOfTime60-89DaysPastDueNotWorse']+weight[0]*df['NumberOfTime30-59DaysPastDueNotWorse']
    
data_sets[0][['LateScore']].head(10) 

## Remove the features we don't need

In [ ]:
print(list(data_sets[0]))

In [ ]:
for df in data_sets:
    df.drop('NumberOfTime30-59DaysPastDueNotWorse', axis=1, inplace=True)
    df.drop('NumberOfTime60-89DaysPastDueNotWorse', axis=1, inplace=True)
    df.drop('NumberOfTimes90DaysLate', axis=1, inplace=True)
    df.drop('NumberOfDependents', axis=1, inplace=True)

In [ ]:
print(list(data_sets[0]))

## Save the data frames as csv files

In [ ]:
for i in range(4):
    data_sets[i].to_csv(filenames[i],index_label='Id')